In [3]:
file_path = "arwiki-20180920-corpus.xml"

with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

print(text[:1000])  # Print first 1000 characters to check


<?xml version="1.0" encoding="UTF-8"?>
<!-- ########################################################## -->
<!-- #                                                          -->
<!-- #  arwiki-20180920.xml                                     -->
<!-- #                                                          -->
<!-- #                                                          -->
<!-- #  (C) Copyright 2018 Wikimedia Foundation, Inc.           -->
<!-- #                        http://ar.wikipedia.org           -->
<!-- #  (C) Copyright this adaption: Peter Kolb                 -->
<!-- #               peter.kolb@linguatools.org                 -->
<!-- #               http://www.linguatools.org/tools/corpora/  -->
<!-- #                                                          -->
<!-- #  This work is made available under the Creative Commons  -->
<!-- #  Attribution-ShareAlike 3.0 License:                     -->
<!-- #  http://creativecommons.org/licenses/by-sa/3.0/legalcode -->
<!-- #   

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer

# Download NLTK Arabic resources
nltk.download('stopwords')

# Initialize Arabic stopwords and stemmer
arabic_stopwords = set(stopwords.words('arabic'))
stemmer = ISRIStemmer()

def preprocess_arabic(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove non-Arabic characters (keep only Arabic letters and spaces)
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize (split by spaces)
    words = text.split()
    
    # Remove stopwords
    words = [word for word in words if word not in arabic_stopwords]
    
    # Apply stemming
    words = [stemmer.stem(word) for word in words]
    
    return ' '.join(words)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
clean_paragraphs = []

file_path = 'arwiki-20180920-corpus.xml' 

MAX_LINES = 10000  # Limit lines for quick testing

with open(file_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i >= MAX_LINES:
            break
        line = line.strip()

        # Skip XML tags, HTML comments, and empty lines
        if line.startswith('<?xml') or line.startswith('<!--') or line.startswith('<') or line == '':
            continue

        # Now clean and save real Arabic text
        clean_line = preprocess_arabic(line)
        
        # Only add non-empty cleaned text
        if clean_line.strip() != '':
            clean_paragraphs.append(clean_line)

In [26]:
print(f" Finished preprocessing! Total cleaned paragraphs: {len(clean_paragraphs)}")

 Finished preprocessing! Total cleaned paragraphs: 814


In [27]:
# Create DataFrame
df = pd.DataFrame({'paragraph': clean_paragraphs})
df = df.fillna('')


In [28]:
# Save cleaned sample
df.to_csv('cleaned_arabic_wiki_sample.csv', index=False, encoding='utf-8')

print(" Cleaned Arabic sample saved successfully to cleaned_arabic_wiki_sample.csv")

 Cleaned Arabic sample saved successfully to cleaned_arabic_wiki_sample.csv


In [29]:
# Print a sample paragraph
if len(clean_paragraphs) > 0:
    print("\n Sample Context:\n", clean_paragraphs[6])


 Sample Context:
 زخر ياه ارض سطح بعد شكل تنع حيي يه، نقص زدد عمق عمد حية ائة بحر حيط وجد ماء كعمل سسي كتل حيوية، ويك عمل حدد ناج كمة قدر غذي نبت ذاب وسف ركب ترج مونيوم نتر ضفة ثني كسد كرب


In [32]:
print(df.columns)


Index(['paragraph'], dtype='object')


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = df['paragraph'].tolist()  # Use all cleaned texts from DataFrame

# Initialize TF-IDF
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(corpus)

print(X.shape)  # Number of documents x number of features
print(tfidf.get_feature_names_out())  # See the vocabulary

# Save cleaned sample
df.to_csv('cleaned_arabic_wiki_sample.csv', index=False, encoding='utf-8')
print("Cleaned Arabic sample saved successfully to cleaned_arabic_wiki_sample.csv")

# Print a sample paragraph
if len(clean_paragraphs) > 6:  # Notice: check if index 6 exists
    print("\nSample Context:\n", clean_paragraphs[6])


(814, 4111)
['آبر' 'آتم' 'آخر' ... 'يوم' 'يون' 'يوه']
Cleaned Arabic sample saved successfully to cleaned_arabic_wiki_sample.csv

Sample Context:
 زخر ياه ارض سطح بعد شكل تنع حيي يه، نقص زدد عمق عمد حية ائة بحر حيط وجد ماء كعمل سسي كتل حيوية، ويك عمل حدد ناج كمة قدر غذي نبت ذاب وسف ركب ترج مونيوم نتر ضفة ثني كسد كرب
